In [21]:
import numpy as np
from itertools import product
import random

In [26]:
class Jogador():
  def __init__(self, tabuleiro, fator_estocastico=0):
    self.tabuleiro = tabuleiro
    self.x = 8
    self.y = 8
    self.fator_estocastico = fator_estocastico

  def ver(self, direcao):
    match direcao:
      case "U":
        return self.tabuleiro[self.y-1, self.x]
      case "D":
        return self.tabuleiro[self.y+1, self.x]
      case "L":
        return self.tabuleiro[self.y, self.x-1]
      case "R":
        return self.tabuleiro[self.y, self.x+1]


  def mover(self, direcao):

    erro = random.random()

    if erro < self.fator_estocastico:
      direcao = random.choice(list('UDLR'))

    if self.ver(direcao) == '.':
      match direcao:
        case "U":
          self.y -= 1
        case "D":
          self.y += 1
        case "L":
          self.x -= 1
        case "R":
          self.x += 1


class Labirinto():
  def __init__(self):
    self.tabuleiro = np.asarray(list("##########"+
                              "#$.......#"+
                              "#........#"+
                              "#........#"+
                              "#........#"+
                              "#........#"+
                              "#........#"+
                              "#........#"+
                              "#........#"+
                              "##########")).reshape((10,10))
    self.player = Jogador(self)


  def render(self):
    display = self.tabuleiro.copy()
    display[self.player.y, self.player.x] = '@'

    print(display)

[['#' '#' '#' '#' '#' '#' '#' '#' '#' '#']
 ['#' '$' '.' '.' '.' '.' '.' '.' '.' '#']
 ['#' '.' '.' '.' '.' '.' '.' '.' '.' '#']
 ['#' '.' '.' '.' '.' '.' '.' '.' '.' '#']
 ['#' '.' '.' '.' '.' '.' '.' '.' '.' '#']
 ['#' '.' '.' '.' '.' '.' '.' '.' '.' '#']
 ['#' '.' '.' '.' '.' '.' '.' '.' '.' '#']
 ['#' '.' '.' '.' '.' '.' '.' '.' '.' '#']
 ['#' '.' '.' '.' '.' '.' '.' '.' '@' '#']
 ['#' '#' '#' '#' '#' '#' '#' '#' '#' '#']]


In [ ]:
""